Install requirements:

pip install -r requirements.txt

Step 1: Load raw invoices and re-arrange info; match up with YDD shipment info and try to assign to Customer ID.

In [7]:
from pathlib import Path
import sys, pandas as pd, traceback
import importlib
import ups_invoice_parser
importlib.reload(ups_invoice_parser)
from ups_invoice_parser import UpsInvLoader, UpsInvNormalizer, UpsCustomerMatcher

FLAG_DEBUG = False  # Set to True to save intermediate Excel files for debugging @ /data/temp

def main():
    # === 1) Select + validate + archive ===
    loader = UpsInvLoader()
    loader.run_import(interactive=True, cli_fallback=False)
    file_list = getattr(loader, "invoices", None)
    if not file_list or not isinstance(file_list, list) or len(file_list) == 0:
        print("❗ No files were selected. Exiting.")
        return
    print(f"📥 Selected {len(file_list)} CSV file(s)")

    # === 2) Normalize invoices ===
    normalizer = UpsInvNormalizer(file_list)
    normalizer.load_invoices()
    normalizer.merge_invoices()
    normalizer.standardize_invoices()
    normalized_df = normalizer.get_normalized_data()
    if FLAG_DEBUG:
        normalized_df.to_excel("data/temp/normalized_invoices.xlsx", index=False)
        print("[Debug] ✅ Normalized invoices saved to data/temp/normalized_invoices.xlsx")
    print(f"✅ Normalized {len(normalized_df)} rows from {len(file_list)} files")

    # === 3) Match customers & classify charges ===
    matcher = UpsCustomerMatcher(normalized_df, use_api=True, ydd_threads=3)
    matcher.match_customers()
    matched_df = matcher.get_matched_data()
    print(f"✅ Matching complete — {matched_df['cust_id'].nunique()} unique customers")

    # Notify user if there are unmapped charges
    unassigned_mask = matched_df["cust_id"].isna() | (matched_df["cust_id"].astype(str).str.strip() == "")
    if unassigned_mask.any():
        print(f"⚠️ {unassigned_mask.sum()} rows still have blank/NaN cust_id")

    # Optionally save the matched_df for step 2
    matched_df.to_pickle("data/temp/matched_invoices.pkl")
    if FLAG_DEBUG:
        matched_df.to_excel("data/temp/matched_invoices.xlsx", index=False)
        print("[Debug] ✅ Matched invoices saved to data/temp/matched_invoices.xlsx")
    print("💾 Matched invoices saved to data/temp/matched_invoices.pkl")

# Directly call main() for notebook usability
try:
    main()
except Exception as e:
    print(f"❌ Error: {e}", file=sys.stderr)
    traceback.print_exc()
    raise

📁 Archived 15 files to \\TRANS-SERVER\Acct2\TWL\UPS\TWL UPS Invoice Parser\data\raw_invoices\355
📥 Selected 15 CSV file(s)
✓ Loaded Invoice_000000HE6132355_083025.csv with encoding utf-8
✓ Loaded Invoice_000000J2158C355_083025.csv with encoding utf-8
✓ Loaded Invoice_000000K5811C355_083025.csv with encoding utf-8


a:\TWL\UPS\TWL UPS Invoice Parser\ups_invoice_parser.py:349: DtypeWarning: Columns (36,195) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


✓ Loaded Invoice_000000XJ3936355_083025.csv with encoding cp1252
✓ Loaded Invoice_000000Y209J6355_083025.csv with encoding utf-8


a:\TWL\UPS\TWL UPS Invoice Parser\ups_invoice_parser.py:349: DtypeWarning: Columns (36,38,104,106,128,195) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


✓ Loaded Invoice_000000H930G4355_083025.csv with encoding utf-8


a:\TWL\UPS\TWL UPS Invoice Parser\ups_invoice_parser.py:349: DtypeWarning: Columns (36,38,104,106,195) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


✓ Loaded Invoice_000000H930G2355_083025.csv with encoding cp1252


a:\TWL\UPS\TWL UPS Invoice Parser\ups_invoice_parser.py:349: DtypeWarning: Columns (36,104,106,128,195) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


✓ Loaded Invoice_000000H930G3355_083025.csv with encoding utf-8
✓ Loaded Invoice_000000G2C794355_083025.csv with encoding utf-8
✓ Loaded Invoice_000000G2G153355_083025.csv with encoding utf-8
✓ Loaded Invoice_000000G2G154355_083025.csv with encoding utf-8
✓ Loaded Invoice_000000G2G156355_083025.csv with encoding utf-8
✓ Loaded Invoice_000000GH3237355_083025.csv with encoding utf-8
✓ Loaded Invoice_000000GH3238355_083025.csv with encoding utf-8
✓ Loaded Invoice_000000GH3239355_083025.csv with encoding utf-8
✓ Loaded Invoice_000000GH3238355_083025.csv with encoding utf-8
✓ Loaded Invoice_000000GH3239355_083025.csv with encoding utf-8
✅ Normalized 104084 rows from 15 files
✅ Normalized 104084 rows from 15 files
[YDD] Login OK in 0.475s (token len=292)
[YDD] Refs total=560, cached=550, querying=10, threads=3, batch_size=9
[YDD] Login OK in 0.475s (token len=292)
[YDD] Refs total=560, cached=550, querying=10, threads=3, batch_size=9
[YDD] ❌ Missing 10 ref(s). Saved to: \\TRANS-SERVER\Acct2\

Before moving to next section, **you have to make sure**:

1. Charges in output/UnmappedCharges.xlsx have been added to data/mappings/Charges.csv

2. Exceptions(shipments not matched in YDD) in output/ExceptionImport_YDD.xlsx are properly matched with customer ID, especially for shipments allocated to "F000222". And then import the template to YDD.

3. If Xero settings have been updated: 

- Check data/mappings/Contacts.csv and data/mappings/InventoryItems-xxxxxxxx.csv from Xero have been updated. (if necessary)

4. If new customer has been added recently:

- Check data/mappings/ARCalculator.csv
- Check data/mappings/Pickups.csv

In [ ]:
from pathlib import Path
import sys, pandas as pd, traceback
import importlib
import ups_invoice_parser
importlib.reload(ups_invoice_parser)
from ups_invoice_parser import UpsInvoiceBuilder, UpsInvoiceExporter

def main():
    # Load matched invoices from step 1
    matched_df = pd.read_pickle("data/temp/matched_invoices.pkl")

    # === 4) Build composite invoice structure ===
    builder = UpsInvoiceBuilder(matched_df)
    builder.build_invoices()
    builder._scc_handler()
    invoices_dict = builder.get_invoices()
    if not invoices_dict:
        raise RuntimeError("No Invoice objects were built — check earlier steps.")
    print(f"✅ Built {len(invoices_dict)} Invoice objects")

    # === 5) Save invoices (.pkl) ===
    builder.save_invoices()

    # === 6) Reload from .pkl ===
    first_invoice = next(iter(invoices_dict.values()))
    batch_number = getattr(first_invoice, "batch_num", None)
    if not batch_number:
        raise RuntimeError("Batch number not available (from invoice).")
    reload_builder = UpsInvoiceBuilder(pd.DataFrame())
    reload_builder.load_invoices(batch_number)
    print(f"✅ Reloaded {len(reload_builder.invoices)} invoices from saved file")

    # === 7) Initialize exporter ===
    exporter = UpsInvoiceExporter(invoices=reload_builder.invoices)

    # === 8) Master export (Details + Summaries + General Cost) ===
    exporter.export()

    # === 9) YiDiDa templates (AP + AR) ===
    exporter.generate_ydd_ap_template()
    exporter.generate_ydd_ar_template()

    # === 10) Xero templates (AP + AR) ===
    exporter.generate_xero_templates()

    # === 11) Per-customer workbooks ===
    exporter.generate_customer_invoices()

    print(f"✅ All exports completed for batch {batch_number}")
    output_folder = Path.cwd() / 'output' / str(batch_number)
    print(f"📁 Output folder: {output_folder}")
    

try:
    main()
except Exception as e:
    print(f"❌ Error: {e}", file=sys.stderr)
    traceback.print_exc()
    raise

✅ Built 15 Invoice objects
📁 Invoices saved to \\TRANS-SERVER\Acct2\TWL\UPS\TWL UPS Invoice Parser\data\raw_invoices\365\invoices_365.pkl
📁 Invoices saved to \\TRANS-SERVER\Acct2\TWL\UPS\TWL UPS Invoice Parser\data\raw_invoices\365\invoices_365.pkl
✅ Invoices loaded from \\TRANS-SERVER\Acct2\TWL\UPS\TWL UPS Invoice Parser\data\raw_invoices\365\invoices_365.pkl
✅ Reloaded 15 invoices from saved file
✅ Loaded Contacts.csv (51 rows)
✅ Loaded InventoryItems-20250831.csv (51 rows)
✅ Invoices loaded from \\TRANS-SERVER\Acct2\TWL\UPS\TWL UPS Invoice Parser\data\raw_invoices\365\invoices_365.pkl
✅ Reloaded 15 invoices from saved file
✅ Loaded Contacts.csv (51 rows)
✅ Loaded InventoryItems-20250831.csv (51 rows)
📁 UPS invoice export saved to \\TRANS-SERVER\Acct2\TWL\UPS\TWL UPS Invoice Parser\output\365\UPS_Invoice_Export.xlsx
📁 YiDiDa AP template saved to \\TRANS-SERVER\Acct2\TWL\UPS\TWL UPS Invoice Parser\output\365\YDD_AP_Template.xlsx
📁 UPS invoice export saved to \\TRANS-SERVER\Acct2\TWL\U